In [1]:
import peachbox

In [2]:
batch_views = peachbox.CassandraDriver()
batch_views.set_keyspace('batch_views')

In [28]:
query = batch_views.execute("SELECT * FROM reviews")

In [29]:
list(query)

[Row(partition_key=u'00cb1793-4206-af11-e58a-c90242ac1100', product_id=u'B0002KPI2S', review_count=1, review_id=u'2420188659791460480', score=5, true_as_of_seconds=911520000),
 Row(partition_key=u'008f9d81-5006-b411-e58a-c90242ac1100', product_id=u'B0013V3GN8', review_count=9, review_id=u'-9099635915169597474', score=4, true_as_of_seconds=900633600),
 Row(partition_key=u'00d876e6-1806-b511-e58a-c90242ac1100', product_id=u'0783221223', review_count=1, review_id=u'-8928698227642725982', score=3, true_as_of_seconds=912816000),
 Row(partition_key=u'00a3b623-c806-ae11-e58a-c90242ac1100', product_id=u'6302799139', review_count=1, review_id=u'9212148428108207140', score=5, true_as_of_seconds=900979200),
 Row(partition_key=u'23db6db0-06b4-11e5-8ac9-0242ac11000c', product_id=u'B007OWUVDS', review_count=16, review_id=u'-2230019442873608950', score=5, true_as_of_seconds=891648000),
 Row(partition_key=u'00a503f1-1006-ae11-e58a-c90242ac1100', product_id=u'6303951619', review_count=9, review_id=u'48

In [23]:
query = batch_views.execute("SELECT true_as_of_seconds, review_count FROM reviews WHERE product_id='B0002KPI2S'")
review_counts = map(lambda x: (x.true_as_of_seconds, x.review_count), query)
                                                                              

In [34]:
review_counts

[(911520000, 1), (911520000, 1), (911520000, 1), (911520000, 1)]

In [35]:
latest_timestamp = max(map(lambda x: x[0], review_counts))
latest_timestamp

911520000

In [42]:
real_time_views.execute("SELECT COUNT(*) FROM reviews WHERE product_id='B00004U12X'\
                            AND true_as_of_seconds>"+str(latest_timestamp)+" ALLOW FILTERING")

[Row(count=1)]

In [41]:
import kafka.client                                                                                                        
import kafka.producer      
import ujson as json
import time
                                                                                                                           
client   = kafka.client.KafkaClient('localhost:9092')                                                                      
producer = kafka.producer.SimpleProducer(client)                                                                           
                                                                                                                           
message = {'user_id':'A37I5QIB*9UMPD',
           'product_id':'B00004U12X',
           'review':'review text',
           'summary':'Summarizing a great movie',
           'profile_name':'ItsMe',
           'helpfulness':'2/2',
           'time':int(time.time()),
           'score':5.0}
                                                                                                   
producer.send_messages('movie_reviews', json.dumps(message))                                                                      


[ProduceResponse(topic='movie_reviews', partition=0, error=0, offset=4680)]